In [11]:
from datetime import datetime # for converting from unix to iso
import json # for pretty printing json dumps
import dateutil.parser as dp # for converting from iso to unix
import folium # for plotting coordinates on a map
from colour import Color # for generating a color scale
import requests # for making a request to OSRM API
import polyline # for converting from polyline to coordinates
from xml.dom import minidom # for parsing gpx files

In [12]:
filename = 'Deaf_Bind_Service_Center.gpx'

In [13]:
# Parsing the .gpx file

xmldoc = minidom.parse(filename)
itemlist = xmldoc.getElementsByTagName('trkpt') # coordinates
timestamps = xmldoc.getElementsByTagName('time') # timestamps
timestamps = timestamps[1:] # ignoring metadata timestamp
print(timestamps[1].firstChild.nodeValue)

2019-01-13T20:02:21Z


In [14]:
# Converting ISO time to Unix (required for OSRM call)

unix = []
for pt in timestamps:
    iso_time = pt.firstChild.nodeValue
    parsed_t = dp.parse(iso_time)
    t_in_seconds = parsed_t.strftime('%s')
    unix.append(t_in_seconds)

In [15]:
# Plotting gpx trace from file

original_coordinates = [] # [(lat, lon),...]
for pt, time in zip(itemlist, unix):
    readable_time = datetime.utcfromtimestamp(int(time)).strftime('%Y-%b-%d %H:%M:%S')
    original_coordinates.append((float(pt.attributes['lat'].value), float(pt.attributes['lon'].value), readable_time))
# print(original_coordinates)

original_map = folium.Map(location=[original_coordinates[0][0], original_coordinates[0][1]], zoom_start=16) # Map centered at first coordinate of route
for point in original_coordinates:
    folium.CircleMarker(
        [point[0], point[1]],
        radius=4,
        tooltip=point[2],
        color = 'b',
        fill_color = 'red'
    ).add_to(original_map)

original_map

In [16]:
# Generating URL for OSRM request

url = "http://127.0.0.1:5000/match/v1/foot/"
for pt in itemlist:
    url += pt.attributes['lon'].value + ","+  pt.attributes['lat'].value +";"
url = url[0:len(url)-1] # removing the last semicolon in the url

# Adding attributes to the request
url += "?tidy=true&overview=full&gaps=ignore"

# Adding time to the request
url += "&timestamps="
for time in unix:
    url += time + ";"
url = url[0:len(url)-1] # removing the last semicolon in the url

In [17]:
# Snapping the trace using OSRM match
# Need to set up OSRM backend to run this part

r = requests.get(url)
res = r.json()
# print(json.dumps(res, indent=2))

In [18]:
decoded_coordinates = [] # snapped coordinates

for item in res['matchings']:
    decoded_coordinates.append((item['confidence'], polyline.decode(item['geometry']), item['duration'], item['distance']))
# decoded_coordinates = [item for sublist in decoded_coordinates for item in sublist]

In [19]:
# Creating a color scale for confidence

colors = list(Color("red").range_to(Color("green"),100))

In [21]:
# Plotting on a folium map

snapped_map = folium.Map(location=decoded_coordinates[0][1][1], zoom_start=16) # Map centered at first coordinate of route
for matching in decoded_coordinates:
    folium.PolyLine(matching[1], 
                    tooltip=str(matching[0]),
                    popup= "Duration: " + str(matching[2]) + "<br>" + "Distance: " + str(matching[3]) + "m",
                    color=colors[int(matching[0]*100)].hex
                   ).add_to(snapped_map)
snapped_map